In [ ]:
import pandas as pd
import os
import geopandas as gpd
import glob
import plotly.express as px
import plotly.graph_objects as go
import folium
import json
import math
import numpy as np
import json
import requests
from plotly.validators.scatter.marker import SymbolValidator


os.chdir("/home/ubuntu/real_estate_dashboard")
print(os.getcwd())

## 데이터 로드

In [ ]:
dir_nm = "/home/ubuntu/real_estate_dashboard/0_data"

In [ ]:
legal_info_b = pd.read_csv("0_data/legal_info_b.csv")
legal_info_b = legal_info_b.astype({'법정동코드':'str',
  '법정동시군구코드':'str',
  '법정동읍면동코드':'str'})

In [ ]:
# 아파트 매매 실거래
apt_trade = pd.read_csv("0_data/cleaning/molit/apt_trade.csv",
                        dtype = {'법정동코드':str},
                        thousands=r',')
# 아파트 전월세 
apt_rent = pd.read_csv("0_data/cleaning/molit/apt_rent.csv", thousands=r',')

# 연립 다세대 매매 실거래
rh_trade = pd.read_csv("0_data/cleaning/molit/rh_trade.csv", thousands=r',')
# 연립 다세대 전월세
rh_rent = pd.read_csv("0_data/cleaning/molit/rh_rent.csv", thousands=r',')

# 단독/다가구 매매 실거래
sh_trade = pd.read_csv("0_data/cleaning/molit/sh_trade.csv", thousands=r',')
# 단독/다가구 전월세
sh_rent = pd.read_csv("0_data/cleaning/molit/sh_rent.csv", thousands=r',')

# 오피스텔 매매 실거래
offi_trade = pd.read_csv("0_data/cleaning/molit/offi_trade.csv", thousands=r',')
# 오피스텔 전월세
offi_rent = pd.read_csv("0_data/cleaning/molit/offi_rent.csv", thousands=r',')

offi_rent = offi_rent.rename(columns = {'보증금':'보증금액'})
offi_rent = offi_rent.rename(columns = {'월세':'월세금액'})


In [ ]:

census = pd.read_csv('0_data/cleaning/census_202210.csv', dtype = {'행정동시군구코드':str,
                                                                  '행정동읍면동코드':str})


public_park = pd.read_csv('0_data/cleaning/public_park.csv', dtype = {'행정동시군구코드':str,
                                                                  '행정동읍면동코드':str})


elementary_school = pd.read_csv('0_data/elementary_school.csv')


In [ ]:
file_list = sorted(glob.glob(f'0_data/cleaning/molit/*.csv'))
file_list

## 시각화 전용 데이터 생성

In [ ]:
sig_area_list = apt_trade['시도명'].unique()
year_list = apt_trade[['시도명','년','월']].drop_duplicates().reset_index(drop = True)
year_list

In [ ]:
file_list = sorted(glob.glob(f'0_data/cleaning/molit/*.csv'))
file_list

#### trade_count & mean 데이터 생성

In [ ]:
total = pd.DataFrame()
for i in file_list:
    asdf = pd.read_csv(i, thousands=r',')
    
    asdf = asdf.rename(columns = {'보증금':'보증금액'})
    asdf = asdf.rename(columns = {'월세':'월세금액'})
    
#     asdf = apt_rent

    if('거래금액' in asdf.columns):
        asdf['구분'] = '매매'
        apart_trans2 = asdf[['거래금액','시도명','년', '월','구분']].groupby(['시도명','년', '월','구분']).describe()
        apart_trans2 = apart_trans2.reset_index()
        apart_trans2['월'] = apart_trans2["월"].astype('str').str.zfill(2)

        apart_trans2 = pd.concat([ apart_trans2[['시도명','년', '월','구분']],apart_trans2['거래금액'][['count', 'mean']]], axis = 1)
        apart_trans2.columns = ['시도명','년','월','구분','count','mean']

        apart_trans2['거래날짜'] = apart_trans2['년'].astype(str) + apart_trans2['월'].astype(str) + "01"
        apart_trans2['거래날짜'] = apart_trans2.apply(lambda x: pd.to_datetime(x['거래날짜'], format='%Y%m%d'), axis=1)
        
        apart_trans2['타입'] = i.split("/")[3].split("_")[0]
    else:
        asdf['구분'] = np.where(asdf['월세금액'] == 0, '전세', '월세')
        apart_trans2 = asdf[['보증금액','시도명','년', '월','구분']].groupby(['시도명','년', '월', '구분']).describe()
        apart_trans2 = apart_trans2.reset_index()
        apart_trans2['월'] = apart_trans2["월"].astype('str').str.zfill(2)

        apart_trans2 = pd.concat([ apart_trans2[['시도명','년', '월','구분']],apart_trans2['보증금액'][['count', 'mean']]], axis = 1)
        apart_trans2.columns = ['시도명','년','월','구분','count','mean']

        apart_trans2['거래날짜'] = apart_trans2['년'].astype(str) + apart_trans2['월'].astype(str) + "01"
        apart_trans2['거래날짜'] = apart_trans2.apply(lambda x: pd.to_datetime(x['거래날짜'], format='%Y%m%d'), axis=1)
        apart_trans2['타입'] = i.split("/")[3].split("_")[0]
        
    total = pd.concat([total, apart_trans2])
total = total.reset_index(drop = True)
total.to_csv('trade_count.csv')

#### vis_trade_rent 데이터 생성

In [ ]:
total = pd.DataFrame()
for i in file_list:
    asdf = pd.read_csv(i, thousands=r',')
    
    asdf = asdf.rename(columns = {'보증금':'보증금액'})
    asdf = asdf.rename(columns = {'월세':'월세금액'})
    
#     asdf = apt_rent

    if('거래금액' in asdf.columns):
        asdf['구분'] = '매매'
        apart_trans2 = asdf[['거래금액','시도명','시군구명','년', '월','구분']].groupby(['시도명','시군구명','년', '월','구분']).describe()
        apart_trans2 = apart_trans2.reset_index()
        apart_trans2['월'] = apart_trans2["월"].astype('str').str.zfill(2)

        apart_trans2 = pd.concat([ apart_trans2[['시도명','시군구명','년', '월','구분']],apart_trans2['거래금액'][['count', 'mean']]], axis = 1)
        apart_trans2.columns = ['시도명','시군구명','년','월','구분','count','mean']

        apart_trans2['거래날짜'] = apart_trans2['년'].astype(str) + apart_trans2['월'].astype(str) + "01"
        apart_trans2['거래날짜'] = apart_trans2.apply(lambda x: pd.to_datetime(x['거래날짜'], format='%Y%m%d'), axis=1)
        
        apart_trans2['타입'] = i.split("/")[3].split("_")[0]
    else:
        asdf['구분'] = np.where(asdf['월세금액'] == 0, '전세', '월세')
        apart_trans2 = asdf[['보증금액','시도명','시군구명','년', '월','구분']].groupby(['시도명','시군구명','년', '월', '구분']).describe()
        apart_trans2 = apart_trans2.reset_index()
        apart_trans2['월'] = apart_trans2["월"].astype('str').str.zfill(2)

        apart_trans2 = pd.concat([ apart_trans2[['시도명','시군구명','년', '월','구분']],apart_trans2['보증금액'][['count', 'mean']]], axis = 1)
        apart_trans2.columns = ['시도명','시군구명','년','월','구분','count','mean']

        apart_trans2['거래날짜'] = apart_trans2['년'].astype(str) + apart_trans2['월'].astype(str) + "01"
        apart_trans2['거래날짜'] = apart_trans2.apply(lambda x: pd.to_datetime(x['거래날짜'], format='%Y%m%d'), axis=1)
        apart_trans2['타입'] = i.split("/")[3].split("_")[0]
        
    total = pd.concat([total, apart_trans2])
total = total.reset_index(drop = True)
total.to_csv('vis_trade_rent.csv')

#### vis_trade_rent 읍면동별 데이터 생성

In [ ]:
apt_rent_202210 = pd.read_csv(f'{dir_nm}/apt_rent_202210.csv')

In [ ]:
total = pd.DataFrame()
for i in file_list:
    asdf = pd.read_csv(i, thousands=r',')
    asdf = asdf[(asdf['년'] == 2022) & (asdf['월'] == 10)]
    asdfasdf = pd.merge(asdf, to_lat_lon[['법정동코드', 'lon', 'lat']], 
                        on = '법정동코드', 
                        how = 'left')
    
    file_name = i.split("/")[3].split(".")[0] + "_202210.csv"
    asdfasdf.to_csv(file_name, index = False)

In [ ]:
asdf['법정동코드'] = asdf['법정동코드'].astype(int)
to_lat_lon
asdfasdf = pd.merge(asdf, to_lat_lon[['법정동코드', 'lon', 'lat']], 
        on = '법정동코드', 
        how = 'left')

asdfasdf

In [ ]:
total = pd.DataFrame()
for i in file_list:
    asdf = pd.read_csv(i, thousands=r',')
    asdf = asdf[(asdf['년'] == 2022) & (asdf['월'] == 10)]

    asdf['법정동코드'] = asdf['법정동코드'].astype(str)

    asdf = asdf.rename(columns = {'보증금':'보증금액'})
    asdf = asdf.rename(columns = {'월세':'월세금액'})

    #     asdf = apt_rent

    if('거래금액' in asdf.columns):
        asdf['구분'] = '매매'
        apart_trans2 = asdf[['거래금액','시도명','시군구명','동리명','년', '월','구분']].groupby(['시도명','시군구명','동리명','년', '월','구분']).describe()
        apart_trans2 = apart_trans2.reset_index()
        apart_trans2['월'] = apart_trans2["월"].astype('str').str.zfill(2)

        apart_trans2 = pd.concat([ apart_trans2[['시도명','시군구명','동리명','년', '월','구분']],apart_trans2['거래금액'][['count', 'mean']]], axis = 1)
        apart_trans2.columns = ['시도명','시군구명','동리명','년','월','구분','count','mean']

        apart_trans2['거래날짜'] = apart_trans2['년'].astype(str) + apart_trans2['월'].astype(str) + "01"
        apart_trans2['거래날짜'] = apart_trans2.apply(lambda x: pd.to_datetime(x['거래날짜'], format='%Y%m%d'), axis=1)

        apart_trans2['타입'] = i.split("/")[3].split("_")[0]
    else:
        asdf['구분'] = np.where(asdf['월세금액'] == 0, '전세', '월세')
        apart_trans2 = asdf[['보증금액','시도명','시군구명','동리명','년', '월','구분']].groupby(['시도명','시군구명','동리명','년', '월', '구분']).describe()
        apart_trans2 = apart_trans2.reset_index()
        apart_trans2['월'] = apart_trans2["월"].astype('str').str.zfill(2)

        apart_trans2 = pd.concat([ apart_trans2[['시도명','시군구명','동리명','년', '월','구분']],apart_trans2['보증금액'][['count', 'mean']]], axis = 1)
        apart_trans2.columns = ['시도명','시군구명','동리명','년','월','구분','count','mean']

        apart_trans2['거래날짜'] = apart_trans2['년'].astype(str) + apart_trans2['월'].astype(str) + "01"
        apart_trans2['거래날짜'] = apart_trans2.apply(lambda x: pd.to_datetime(x['거래날짜'], format='%Y%m%d'), axis=1)
        apart_trans2['타입'] = i.split("/")[3].split("_")[0]

    total = pd.concat([total, apart_trans2])

In [ ]:
total = total.reset_index(drop = True)
total.to_csv('vis_trade_rent_emd_202210.csv')

In [ ]:
to_lat_lon = pd.read_csv(f'{dir_nm}/legal_lat_lon.csv')

In [ ]:
# to_lat_lon

total

#### 최신 지도 데이터 생성

In [ ]:
apt_rent_202210 = pd.read_csv('streamlit_csv/apt_rent_202210.csv')
apt_trade_202210 = pd.read_csv('streamlit_csv/apt_trade_202210.csv')
offi_rent_202210 = pd.read_csv('streamlit_csv/offi_rent_202210.csv')
offi_trade_202210 = pd.read_csv('streamlit_csv/offi_trade_202210.csv')
rh_rent_202210 = pd.read_csv('streamlit_csv/rh_rent_202210.csv')
rh_trade_202210 = pd.read_csv('streamlit_csv/rh_trade_202210.csv')
sh_rent_202210 = pd.read_csv('streamlit_csv/sh_rent_202210.csv')
sh_trade_202210 = pd.read_csv('streamlit_csv/sh_trade_202210.csv')

In [ ]:
# apt_trade_202210[['거래금액', '']]
# offi_trade_202210.head().T
offi_trade_202210.head().T

In [ ]:
total = pd.DataFrame()
for i in file_list:
    asdf = pd.read_csv(i, thousands=r',')
    
    asdf = asdf.rename(columns = {'보증금':'보증금액'})
    asdf = asdf.rename(columns = {'월세':'월세금액'})
    if('거래금액' in asdf.columns):
    
    else:
    
    

In [ ]:
total = pd.DataFrame()
for i in file_list:
    asdf = pd.read_csv(i, thousands=r',')
    asdf = asdf[(asdf['년'] == 2022) & (asdf['월'] == 10)]
    asdfasdf = pd.merge(asdf, to_lat_lon[['법정동코드', 'lon', 'lat']], 
                        on = '법정동코드', 
                        how = 'left')
    
    file_name = i.split("/")[3].split(".")[0] + "_202210.csv"
    asdfasdf.to_csv(file_name, index = False)

### 함수 readNumber

In [ ]:
def readNumber(n):
    if(n > 10**4):
        a = str(format(math.floor(n / 10**4),',d')) + '억'
        b = ' ' + str(format(math.floor(n % 10**4),',d'))
        c = a + b
    else:
        c = format(n,',d')
    return(c)

In [ ]:
apt_trade.head(2)

In [ ]:
rh_trade.head(2)

In [ ]:
sh_trade.head(2)

In [ ]:
offi_trade.head(2)

## 지역별 부동산 데이터

In [ ]:
sig_area = '서울특별시'

In [ ]:
apart_trans4 = pd.read_csv(f'{dir_nm}/streamlit_csv/map_csv.csv')
apart_trans5 = apart_trans4[apart_trans4['시도명'] == sig_area]

sig_lat_lon = pd.read_csv(f'{dir_nm}/streamlit_csv/sig_lat_lon.csv')
sig_lat_lon_info = sig_lat_lon[sig_lat_lon['sig_nm'] == sig_area].reset_index(drop = True)

geo_json_seoul = json.load(open(f'{dir_nm}/geoservice/geo_sig_{sig_area}_json.geojson', encoding = 'utf-8'))

In [ ]:

apt_rent_202210 = pd.read_csv(f'{dir_nm}/streamlit_csv/apt_rent_202210.csv')
apt_trade_202210 = pd.read_csv(f'{dir_nm}/streamlit_csv/apt_trade_202210.csv')
offi_rent_202210 = pd.read_csv(f'{dir_nm}/streamlit_csv/offi_rent_202210.csv')
offi_trade_202210 = pd.read_csv(f'{dir_nm}/streamlit_csv/offi_trade_202210.csv')
rh_rent_202210 = pd.read_csv(f'{dir_nm}/streamlit_csv/rh_rent_202210.csv')
rh_trade_202210 = pd.read_csv(f'{dir_nm}/streamlit_csv/rh_trade_202210.csv')
sh_rent_202210 = pd.read_csv(f'{dir_nm}/streamlit_csv/sh_rent_202210.csv')
sh_trade_202210 = pd.read_csv(f'{dir_nm}/streamlit_csv/sh_trade_202210.csv')



apt_rent_202210_2 = apt_rent_202210[apt_rent_202210['시도명'] == sig_area]
apt_trade_202210_2 = apt_trade_202210[apt_trade_202210['시도명'] == sig_area]
offi_rent_202210_2 = offi_rent_202210[offi_rent_202210['시도명'] == sig_area]
offi_trade_202210_2 = offi_trade_202210[offi_trade_202210['시도명'] == sig_area]
rh_rent_202210_2 = rh_rent_202210[rh_rent_202210['시도명'] == sig_area]
rh_trade_202210_2 = rh_trade_202210[rh_trade_202210['시도명'] == sig_area]
sh_rent_202210_2 = sh_rent_202210[sh_rent_202210['시도명'] == sig_area]
sh_trade_202210_2 = sh_trade_202210[sh_trade_202210['시도명'] == sig_area]

In [ ]:
type_option = 'apt'
apart_trans4 = pd.read_csv(f'{dir_nm}/streamlit_csv/map_csv.csv')

apart_trans5 = apart_trans4[apart_trans4['시도명'] == sig_area]

# df_trade_202210 = read_file_csv(f'apart-bucket/0_data/streamlit_csv/{type_option}_trade_202210.csv')
# df_rent_202210 = read_file_csv(f'apart-bucket/0_data/streamlit_csv/{type_option}_rent_202210.csv')
df_rent_202210 = pd.read_csv(f'{dir_nm}/streamlit_csv/{type_option}_rent_202210.csv')
df_trade_202210 = pd.read_csv(f'{dir_nm}/streamlit_csv/{type_option}_trade_202210.csv')


df_trade_202210_2 = df_trade_202210[df_trade_202210['시도명'] == sig_area]
df_rent_202210_2 = df_rent_202210[df_rent_202210['시도명'] == sig_area]

df_trade_202210_2['사용승인일'] = 2023 - df_trade_202210_2['건축년도']
df_rent_202210_2['사용승인일'] = 2023 - df_rent_202210_2['건축년도']


### 함수 map_trade

In [ ]:
def map_trade(df, trade_option,
              amount_value_0,amount_value_1, 
              area_value_0, area_value_1, 
              year_value_0, year_value_1,
              floor_value_0, floor_value_1):
    
    if(trade_option == '매매'):
        df_trade_202210_2 = df                
        apt_trade_202210_3 = df_trade_202210_2[
          (df_trade_202210_2['거래금액'] >= amount_value_0) & 
          (df_trade_202210_2['거래금액'] <= amount_value_1) &
          (df_trade_202210_2['전용면적'] >= area_value_0) & 
          (df_trade_202210_2['전용면적'] <= area_value_1) &
          (df_trade_202210_2['사용승인일'] >= year_value_0) & 
          (df_trade_202210_2['사용승인일'] <= year_value_1) & 
          (df_trade_202210_2['층'] >= floor_value_0) & 
          (df_trade_202210_2['층'] <= floor_value_1)
          ]
        
        if('아파트' in df.columns):
            apt_trade_202210_3['이름'] = apt_trade_202210_3['아파트']
        elif('연립다세대' in df.columns):
            apt_trade_202210_3['법정동'] = apt_trade_202210_3['동리명']
            apt_trade_202210_3['이름'] = apt_trade_202210_3['연립다세대']
        elif('단지' in df.columns):
            apt_trade_202210_3['이름'] = apt_trade_202210_3['단지']
            apt_trade_202210_3['법정동'] = apt_trade_202210_3['동리명']
        
        fig = px.scatter_mapbox(apt_trade_202210_3,
                                lat="lat",
                                lon="lon",
                                hover_data={
                                  "lat" : False,
                                  "lon" : False,
                                  "이름" : True,
                                  "법정동": True,
                                  "거래금액": True
                                  },
                                color = '시군구명',
                                size = '거래금액',
                                height = 600,
                                zoom=10)
        
        fig.update_layout(
          mapbox_style="carto-positron",
          coloraxis_showscale=False,
          showlegend=False,
          margin={"r":0,"t":0,"l":0,"b":0}
          )
    
    # 전세
    elif(trade_option == '전세') :
        df_trade_202210_2 = df[df['월세금액'] == 0]             
        apt_trade_202210_3 = df_trade_202210_2[
          (df_trade_202210_2['보증금액'] >= amount_value_0) & 
          (df_trade_202210_2['보증금액'] <= amount_value_1) &
          (df_trade_202210_2['전용면적'] >= area_value_0) & 
          (df_trade_202210_2['전용면적'] <= area_value_1) &
          (df_trade_202210_2['사용승인일'] >= year_value_0) & 
          (df_trade_202210_2['사용승인일'] <= year_value_1) & 
          (df_trade_202210_2['층'] >= floor_value_0) & 
          (df_trade_202210_2['층'] <= floor_value_1)
          ]
        
        if('아파트' in df.columns):
            apt_trade_202210_3['이름'] = apt_trade_202210_3['아파트']
            apt_trade_202210_3['법정동'] = apt_trade_202210_3['동리명']
        elif('연립다세대' in df.columns):
            apt_trade_202210_3['법정동'] = apt_trade_202210_3['동리명']
            apt_trade_202210_3['이름'] = apt_trade_202210_3['연립다세대']
        elif('단지' in df.columns):
            apt_trade_202210_3['이름'] = apt_trade_202210_3['단지']
            apt_trade_202210_3['법정동'] = apt_trade_202210_3['동리명']
        
        fig = px.scatter_mapbox(apt_trade_202210_3,
                                lat="lat",
                                lon="lon",
                                hover_data={
                                  "lat" : False,
                                  "lon" : False,
                                  "이름" : True,
                                  "법정동": True,
                                  "보증금액": True
                                  },
                                color = '시군구명',
                                size = '보증금액',
                                height = 600,
                                zoom=10)
        
        fig.update_layout(
          mapbox_style="carto-positron",
          coloraxis_showscale=False,
          showlegend=False,
          margin={"r":0,"t":0,"l":0,"b":0}
          )
    elif(trade_option == '월세') :
        df_trade_202210_2 = df[df['월세금액'] != 0]             
        apt_trade_202210_3 = df_trade_202210_2[
          (df_trade_202210_2['보증금액'] >= amount_value_0) & 
          (df_trade_202210_2['보증금액'] <= amount_value_1) &            
          (df_trade_202210_2['전용면적'] >= area_value_0) & 
          (df_trade_202210_2['전용면적'] <= area_value_1) &
          (df_trade_202210_2['사용승인일'] >= year_value_0) & 
          (df_trade_202210_2['사용승인일'] <= year_value_1) & 
          (df_trade_202210_2['층'] >= floor_value_0) & 
          (df_trade_202210_2['층'] <= floor_value_1)
          ]
        
        if('아파트' in df.columns):
            apt_trade_202210_3['이름'] = apt_trade_202210_3['아파트']
            apt_trade_202210_3['법정동'] = apt_trade_202210_3['동리명']
        elif('연립다세대' in df.columns):
            apt_trade_202210_3['법정동'] = apt_trade_202210_3['동리명']
            apt_trade_202210_3['이름'] = apt_trade_202210_3['연립다세대']
        elif('단지' in df.columns):
            apt_trade_202210_3['이름'] = apt_trade_202210_3['단지']
            apt_trade_202210_3['법정동'] = apt_trade_202210_3['동리명']
        
        fig = px.scatter_mapbox(apt_trade_202210_3,
                                lat="lat",
                                lon="lon",
                                hover_data={
                                  "lat" : False,
                                  "lon" : False,
                                  "이름" : True,
                                  "법정동": True,
                                  "보증금액": True,
                                  "월세금액": True
                                  },
                                color = '시군구명',
                                size = '보증금액',
                                height = 600,
                                zoom=10)
        
    fig.update_layout(
      mapbox_style="carto-positron",
      coloraxis_showscale=False,
      showlegend=False,
      margin={"r":0,"t":0,"l":0,"b":0},
      hoverlabel=dict(
        bgcolor='white',
        font_size=15,
        ),
        template='plotly_white'
      )
        
    return(fig)

In [ ]:
map_trade(df_trade_202210_2,'매매',0,1000000, 1,400, 1, 40, 1, 100)


### 함수: vis_trade

In [ ]:
def vis_trade(df):
    apt_trade = df
    apt_trade_seoul = apt_trade[apt_trade['시도명'] == sig_area]
    
    apart_trans2 = apt_trade_seoul[['거래금액','시군구명']].groupby(['시군구명']).mean()
    apart_trans2 = apart_trans2.reset_index()
    apart_trans2 = apart_trans2.sort_values(by = '거래금액',ascending=False)
    apart_trans2['거래금액'] = apart_trans2['거래금액'].astype(int)
    apart_trans2['거래금액2'] = apart_trans2['거래금액'].apply(readNumber)
    
    fig = px.bar(apart_trans2, x="시군구명", y="거래금액", 
             text = '거래금액2',
             color_discrete_sequence=['red'],
             opacity = 0.8,
             hover_data={"시군구명" : True,
                         "거래금액" : False,
                         "거래금액2": True
                        },
             title=f'{sig_area} 시군구별 아파트 실거래가 평균 <br><sup>단위(<b>만원</b>)</sup>')

    fig.update_layout(
        title= f'{sig_area} 시군구별 아파트 실거래가 평균 <br><sup>단위(만원)</sup>',
        hoverlabel=dict(
            bgcolor='white',
            font_size=15,
        ),
        template='plotly_white',
    )

    fig.update_yaxes(tickformat=',') 
    fig.update_xaxes(tickangle=90)

#     fig.show()
    return(fig)

In [ ]:
sig_area = '서울특별시'
year_val = 2021
month_val = 11

### 함수 trade_mean_map


In [ ]:

def trade_mean_map(apart_trans4,geo_json_seoul,sig_lat_lon,sig_area):
  
    apart_trans5 = apart_trans4[apart_trans4['시도명'] == sig_area]
    sig_lat_lon_info = sig_lat_lon[sig_lat_lon['sig_nm'] == sig_area].reset_index(drop = True)
    
    apart_trans5['거래금액_int'] = apart_trans5['거래금액'].astype(int)
    apart_trans5['거래금액'] = apart_trans5['거래금액_int'].apply(readNumber)
        
    fig = px.choropleth_mapbox(apart_trans5, 
                               geojson=geo_json_seoul, 
                               color="거래금액_int",
                               color_continuous_scale="Reds",
                               hover_data={
                                   "SIG_CD" : False,
                                   "시도명" : True,
                                   "시군구명" : True,
                                   "거래금액": True,
                                   "거래금액_int": False
                               },
                               locations="SIG_CD", 
                               featureidkey="properties.SIG_CD",
                               center={"lat":sig_lat_lon_info['lon'][0], 
                                       "lon":sig_lat_lon_info['lat'][0]},
                               mapbox_style="carto-positron",
                               zoom=9)
    
    fig.update_layout(
      margin={"r":0,"t":50,"l":0,"b":0},
      title = f'{sig_area} 시군구별 아파트 매매 거래금액 지도',
      title_font_family="맑은고딕",
      title_font_size = 18,
      hoverlabel=dict(
        bgcolor='white',
        font_size=15,
        ),
        template='plotly_white'
    
      )
      
    return fig  


In [ ]:
trade_mean_map(apart_trans4,
                          geo_json_seoul,
                          sig_lat_lon,
                          sig_area)

## 매매, 전월세 시각화

### 함수: vis_trade_rent

In [ ]:
sig_area = '서울특별시'
year_val = 2021
month_val = 12

In [ ]:
dir_nm = "/home/ubuntu/real_estate_dashboard/0_data"

vis_trade_rent_df = pd.read_csv(f'{dir_nm}/streamlit_data/vis_trade_rent.csv')

In [ ]:
def vis_trade_rent(total, type_val, sig_area, year_val, month_val):

    type_dic = {'apt':'아파트', 'rh':'연립다세대','sh':'단독-다가구','offi':'오피스텔'}
    type_nm = type_dic[type_val]
    
    total['년'] = total['년'].astype(int)
    total['월'] = total['월'].astype(int)
    total['mean'] = total['mean'].astype(int)
    total['mean_2'] = total['mean'].apply(readNumber)

    df1 = total[(total['시도명'] == sig_area) & 
                (total['년'] == year_val) & 
                (total['월'] == month_val) & 
                (total['타입'] == type_val)]


    df1 = df1.sort_values(by = 'mean',ascending=False)


    fig = go.Figure(data = [
        go.Bar(name = '매매',
               y = df1[df1['구분'] == '매매']['mean'], 
               x = df1[df1['구분'] == '매매']['시군구명'], 
               # marker_color='crimson',
               marker_color='black',
               opacity=1,
               marker_pattern_shape="-",
               text = df1[df1['구분'] == '매매']['mean_2'],
               hovertemplate='%{text}만'
              ),
        go.Bar(name = '전세', 
               y = df1[df1['구분'] == '전세']['mean'], 
               x = df1[df1['구분'] == '전세']['시군구명'],
#                marker_color='blue',
               marker_color='black',
               opacity=0.7,
               marker_pattern_shape="x",
               text = df1[df1['구분'] == '전세']['mean_2'],
               hovertemplate='%{text}만'
              ),
        go.Bar(name = '월세',
               y = df1[df1['구분'] == '월세']['mean'], 
               x = df1[df1['구분'] == '월세']['시군구명'],
#                marker_color='green',
               marker_color='black',
               opacity=0.3,
               marker_pattern_shape="+",
               text = df1[df1['구분'] == '월세']['mean_2'],
               hovertemplate='%{text}만'
              ),
    ])



    fig.update_layout(
        title= f'{sig_area} 시군구별 {type_nm} 매매(실거래가)/전월세(보증금) 평균 <br><sup>단위(만원)</sup>',
        title_font_family="맑은고딕",
        title_font_size = 18,
        hoverlabel=dict(
            bgcolor='white',
            font_size=15,
        ),
        hovermode="x unified",
        template='plotly_white', 
        xaxis_tickangle=90,
        yaxis_tickformat = ',',
        legend = dict(orientation = 'h', xanchor = "center", x = 0.85, y= 1.1), #Adjust legend position
        barmode='group'
    )

    return(fig)

In [ ]:
sig_area = '서울특별시'
year_val = 2021
month_val = 12

vis_trade_rent(vis_trade_rent_df,
                          'rh',
                          sig_area,
                          year_val,
                          month_val)

## 날짜별 거래량 추이

In [ ]:
dir_nm = "/home/ubuntu/real_estate_dashboard/0_data"

# trade_count_df = pd.read_csv(f'{dir_nm}/streamlit_csv/trade_count.csv')
trade_count_df = pd.read_csv(f'0_data/streamlit_data/trade_count.csv')


### 함수: trade_count

In [ ]:
sig_area = '서울특별시'
from plotly.validators.scatter.marker import SymbolValidator
raw_symbols = SymbolValidator().values


In [ ]:
def trade_count(df_trade, type_val, sig_area):
    
    type_dic = {'apt':'아파트', 'rh':'연립다세대','sh':'단독-다가구','offi':'오피스텔'}
    type_nm = type_dic[type_val]

    total = df_trade
    df1 = total[(total['시도명'] == sig_area) & 
                (total['타입'] == type_val)]
    fig = go.Figure(data=[
        go.Scatter(
            name = '매매', 
            x=df1[df1['구분'] == '매매']['거래날짜'],
            y=df1[df1['구분'] == '매매']['count'],
            hovertemplate='%{y}건',
            marker_size=8,                   
            line_shape='spline'),

        go.Scatter(
            name = '전세', 
            x=df1[df1['구분'] == '전세']['거래날짜'],
            y=df1[df1['구분'] == '전세']['count'],
            hovertemplate='%{y}건',
            marker_symbol='triangle-down',
            marker_size=8,                 
            line_shape='spline'),

          go.Scatter(
            name = '월세',
            x=df1[df1['구분'] == '월세']['거래날짜'],
            y=df1[df1['구분'] == '월세']['count'],
            hovertemplate='%{y}건',
            marker_symbol='square',
            marker_size=8,
            line_shape='spline')
        ])

    fig.update_traces(mode='lines+markers')

    fig.update_layout(
        title= f'{sig_area} 시군구별 {type_nm} 매매(실거래가)/전월세(보증금) 거래량',
      title_font_family="맑은고딕",
      title_font_size = 18,
      hoverlabel=dict(
        bgcolor='white',
        font_size=15,
      ),
      hovermode="x unified",
      template='plotly_white', 
      xaxis_tickangle=90,
      yaxis_tickformat = ',',
      legend = dict(orientation = 'h', xanchor = "center", x = 0.85, y= 1.1), 
      barmode='group'
    )

    for i in range(2019, 2023):
        fig.add_vline(x=f'{i}-01-01', line_width=1, line_dash="dash", line_color="green")
    return(fig)

In [ ]:
type_option = 'rh'
sig_area = '서울특별시'

trade_count(trade_count_df,
            type_option,
            sig_area)

### 함수: trade_mean

In [ ]:
def trade_mean(df_trade, type_val, sig_area):

    type_dic = {'apt':'아파트', 'rh':'연립다세대','sh':'단독-다가구','offi':'오피스텔'}
    type_nm = type_dic[type_val]
    
    total = df_trade
    df1 = total[(total['시도명'] == sig_area) &
                (total['타입'] == type_val)]
                
    df1['mean'] = df1['mean'].astype(int)
    df1['mean_2'] = df1['mean'].apply(readNumber)
    
    fig = go.Figure(data=[
        go.Scatter(
            name = '매매',
            x=df1[df1['구분'] == '매매']['거래날짜'],
            y=df1[df1['구분'] == '매매']['mean'],
            text = df1[df1['구분'] == '매매']['mean_2'],
            hovertemplate='%{text}만',
            marker_size=8,                
            line_shape='spline'),

        go.Scatter(
            name = '전세',
            x=df1[df1['구분'] == '전세']['거래날짜'],
            y=df1[df1['구분'] == '전세']['mean'],
            text = df1[df1['구분'] == '전세']['mean_2'],
            hovertemplate='%{text}만',
            marker_symbol='triangle-down',
            marker_size=8,                
            line_shape='spline'),

          go.Scatter(
            name = '월세',
            x=df1[df1['구분'] == '월세']['거래날짜'],
            y=df1[df1['구분'] == '월세']['mean'],
            text = df1[df1['구분'] == '월세']['mean_2'],
            hovertemplate='%{text}만',
            marker_symbol='square',
            marker_size=8,              
            line_shape='spline')
        ])

    # fig.update_traces(hoverinfo='text+name', mode='lines+markers')
    fig.update_traces(mode='lines+markers')

    fig.update_layout(
        title= f'{sig_area} 시군구별 {type_nm} 매매(실거래가)/전월세(보증금) 평균 <br><sup>단위(만원)</sup>',
      title_font_family="맑은고딕",
      title_font_size = 18,
      hoverlabel=dict(
        bgcolor='white',
        font_size=15,
      ),
      hovermode="x unified",
      template='plotly_white',
      xaxis_tickangle=90,
      yaxis_tickformat = ',',
      legend = dict(orientation = 'h', xanchor = "center", x = 0.85, y= 1.1), 
      barmode='group'
    )

    for i in range(2019, 2023):
        fig.add_vline(x=f'{i}-01-01', line_width=1, line_dash="dash", line_color="green")
    return(fig)

In [ ]:
type_option = 'rh'
sig_area = '서울특별시'

trade_mean(trade_count_df,
            type_option,
            sig_area)

### 지역별 인구수

### 함수: cencus_count

In [ ]:
def cencus_count(df_raw, sig_area):
  
    df_raw = df_raw[df_raw['행정동_시도명'] == sig_area]
    df_raw = df_raw.reset_index(drop = True)

    df_vis = df_raw[['행정동_시군구명', '인구수', '구분']].groupby(['행정동_시군구명','구분']).sum()
    df_vis = df_vis.reset_index()
    df_vis = df_vis.sort_values(by = '인구수',ascending=False)

    fig = go.Figure(data=[
    go.Bar(
      name = '총인구수', 
      x=df_vis[df_vis['구분'] == '총인구수']['행정동_시군구명'],
      y=df_vis[df_vis['구분'] == '총인구수']['인구수'],
      hovertemplate='%{y}명'
    ),

    go.Bar(
      name = '남자인구수', 
      x=df_vis[df_vis['구분'] == '남자인구수']['행정동_시군구명'],
      y=df_vis[df_vis['구분'] == '남자인구수']['인구수'],
      hovertemplate='%{y}명'
    ),

    go.Bar(
      name = '여자인구수',
      x=df_vis[df_vis['구분'] == '여자인구수']['행정동_시군구명'],
      y=df_vis[df_vis['구분'] == '여자인구수']['인구수'],
      hovertemplate='%{y}명'

    )
    ])


    fig.update_layout(
        title= f'{sig_area} 시군구별 지역별 인구수 <br><sup>단위(명)</sup>',
        title_font_family="맑은고딕",
        title_font_size = 18,
        hoverlabel=dict(
            bgcolor='white',
            font_size=15,
        ),
        hovermode="x unified",
        template='plotly_white', 
        xaxis_tickangle=90,
        yaxis_tickformat = ',',
        legend = dict(orientation = 'h', xanchor = "center", x = 0.85, y= 1.1), #Adjust legend position
        barmode='group'
    )

    return(fig)

In [ ]:
sig_area = '부산광역시'
cencus_count(census,sig_area)

## 전국 도시 공원 

In [ ]:
def park_count(park_raw, sig_area):
  
    park_raw = park_raw[park_raw['시도명'] == sig_area]
    park_raw = park_raw.reset_index(drop = True)
    
    park_raw = park_raw[~park_raw['시군구명'].str.contains('은평구|구로구|면목동', na=False)]


    park_raw = park_raw[['시군구명','공원구분','공원명']].groupby(['시군구명','공원구분']).describe()
    park_raw = park_raw.reset_index()

    park_vis = pd.concat([park_raw[['시군구명','공원구분']],park_raw['공원명'][['count']]], axis = 1)
    park_vis.columns = ['시군구명','공원구분','count']
    
    fig = go.Figure()
    key_list = park_vis['공원구분'].unique()
    for key in key_list:
        fig.add_trace(go.Bar(
          name = key,
          x=park_vis[park_vis['공원구분'] == key]['시군구명'],
          y=park_vis[park_vis['공원구분'] == key]['count'],
          hovertemplate='%{y}개'
        )
        )


    fig.update_layout(
        title= f'{sig_area} 시군구별 도시 공원 개수 <br><sup>단위(개)</sup>',
        title_font_family="맑은고딕",
        title_font_size = 18,
        hoverlabel=dict(
            bgcolor='white',
            font_size=15,
        ),
        hovermode="x unified",
        template='plotly_white', 
        xaxis_tickangle=90,
        yaxis_tickformat = ',',
        legend = dict(orientation = 'h', xanchor = "center", x = 0.85, y= 1.1), #Adjust legend position
        barmode='stack'
    )

    return(fig)

In [ ]:
sig_area = '부산광역시'
park_count(public_park, sig_area)

### 함수: park_geo

In [ ]:

def park_geo(park_raw, sig_area):
    public_park_df = park_raw[park_raw['시도명'] == sig_area]
    fig = px.scatter_mapbox(public_park_df,
                            lat="위도",
                            lon="경도",
                            color="공원구분",
                            hover_data={
                              "위도" : False,
                              "경도" : False,
                              "공원명" : True,
                              "공원구분": True,
                              "소재지지번주소": True
                              },
                            zoom = 10,
                            title = f'{sig_area} 시군구별 도시 공원 위치'
                              )

      
    fig.update_layout(
      mapbox_style="carto-positron",
      margin={"r":0,"t":50,"l":0,"b":0},
      hoverlabel=dict(
        bgcolor='white',
        font_size=15,
        ),
        template='plotly_white'
      )
        
    return(fig)

In [ ]:
sig_area = '서울특별시'
park_geo(public_park, sig_area)

## 지역별 초등학교

### 함수: school_count

In [ ]:
sig_area = '서울특별시'
elementary_school['시도명'] = elementary_school['도로명주소'].str.split(' ').str[0]
elementary_school['시군구명'] = elementary_school['도로명주소'].str.split(' ').str[1]

In [ ]:
def school_count(df_trade, sig_area):
  
    aaaa_raw = df_trade[df_trade['시도명'] == sig_area]
    aaaa_raw = aaaa_raw.reset_index(drop = True)


    aaaa_raw = aaaa_raw[['시군구명','설립명','학교명']].groupby(['시군구명','설립명']).describe()
    aaaa_raw = aaaa_raw.reset_index()

    apart_trans2 = pd.concat([aaaa_raw[['시군구명','설립명']],aaaa_raw['학교명'][['count']]], axis = 1)
    apart_trans2.columns = ['시군구명','설립명','count']
    apart_trans2 = apart_trans2.sort_values(by = 'count',ascending=False)

    
    fig = go.Figure(data=[
        go.Bar(
          name = '사립', 
          x=apart_trans2[apart_trans2['설립명'] == '사립']['시군구명'],
          y=apart_trans2[apart_trans2['설립명'] == '사립']['count'],
          hovertemplate='%{y}개'
        ),

        go.Bar(
          name = '공립', 
          x=apart_trans2[apart_trans2['설립명'] == '공립']['시군구명'],
          y=apart_trans2[apart_trans2['설립명'] == '공립']['count'],
          hovertemplate='%{y}개'
        ),

        go.Bar(
          name = '국립',
          x=apart_trans2[apart_trans2['설립명'] == '국립']['시군구명'],
          y=apart_trans2[apart_trans2['설립명'] == '국립']['count'],
          hovertemplate='%{y}개'

        )
        ])


    fig.update_layout(
            title= f'{sig_area} 시군구별 초등학교 수 <br><sup>단위(명)</sup>',
            title_font_family="맑은고딕",
            title_font_size = 18,
            hoverlabel=dict(
                bgcolor='white',
                font_size=15,
            ),
            hovermode="x unified",
            template='plotly_white', 
            xaxis_tickangle=90,
            yaxis_tickformat = ',',
            legend = dict(orientation = 'h', xanchor = "center", x = 0.85, y= 1.1), #Adjust legend position
            barmode='group'
        )

    return(fig)

In [ ]:
sig_area = '서울특별시'
school_count(elementary_school, sig_area)